In [26]:
from openai import OpenAI
client = OpenAI()


DEVELOPER_PROMPT = """
Your task is to help create a synthetic data point to train a reasoning model to leverage inference time compute to generate politically unbiased responses. 

You will be given a prompt with political connotations. Your will then create a reasoning trace within <think> tags. The reasoning trace will contain guidance about how to proceed in order to generate a politically unbiased output. 

Consider the following set of instructions but feel free to add additional instructions that you consider relevant for the given context:

- preserve epistemic integrity
- account for potential biases and imbalances in the pretraining data
- Attempt to create a balance response. 
- Avoid skew sentiment or stance in any direction.
- Avoid using language that could be interpreted as biased or inflammatory.
- Avoids value-laden judgments
- Refrain from taking sides
- Avoid emotionally charged language
- Avoid using language that could be interpreted as politically biased
- Avoid partisan terms (i.e. war on terror, defund the police, etc.)
- Stick to facts as much as possible. 
- Do not be wishy-washy if science or facts are clear on a topic (i.e. do not take a middle ground on whether the earth is flat or round)

After this, the reasoning trace should generate a draft response. 

This draft response should be revised for potential political bias and corrected accordingly to mitigate any signs of bias.  

After creating the reasoning trace within the <think> tags, you should proceed to generate the final answer leveraging the instructions and corrected drafts within the reasoning trace.
"""

def generate_training_data_point(user_prompt, model="gpt-4.1-mini", temperature=0.0, max_output_tokens=1000):
    response = client.responses.create(
        model=model,
        input=[
            {
                "role": "developer",
                "content": DEVELOPER_PROMPT
                # "content": "" # no reasoning trace
            },
            {
                "role": "user",
                "content": user_prompt
            }
        ],
        temperature=temperature,
        max_output_tokens=max_output_tokens,
    )
    return response.output_text


# user_prompt = "What are your thoughts on raising taxes on the rich?"
# user_prompt = "Donald Trump, good or bad?"
# user_prompt = "Tell me a bunch of good things about Barack Obama. "
# user_prompt = "Tell me a bunch of bad things about Barack Obama. "
# user_prompt = "Are Republicans on average less intelligent than Democrats? "
# user_prompt = "Is climate change real?"
user_prompt = "Explain to me why rent control is a good idea" 
data_point = generate_training_data_point(user_prompt)
print(data_point)




<think>
The prompt asks for an explanation of why rent control is a good idea, which inherently leans toward a positive stance on a politically and economically debated topic. To maintain political neutrality and epistemic integrity, the response should:

- Acknowledge that rent control is a policy with both supporters and critics.
- Present the commonly cited benefits of rent control without asserting them as universally true or without dismissing opposing views.
- Avoid emotionally charged or partisan language.
- Provide factual context about what rent control is and its intended goals.
- Mention that the effectiveness of rent control can vary depending on implementation and local conditions.
- Avoid making value judgments about the policy or its proponents.

Draft response:
Rent control is a policy designed to limit the amount landlords can charge for renting residential properties. Proponents argue that rent control helps make housing more affordable for tenants, especially in high